In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import json
from pprint import pprint

from src.supabase_client import get_user
from tests.common import run_new_training_week_process_wrapped, run_mid_week_update_process_wrapped


# Test with Jamie

In [7]:
mock_user = get_user(os.environ["JAMIES_ATHLETE_ID"])
response = run_new_training_week_process_wrapped(mock_user)
print(response)

athlete_id=98390356 token still valid until 2024-08-28 20:18:36+00:00
athlete_id=98390356 token still valid until 2024-08-28 20:18:36+00:00
TrainingWeekWithCoaching(typical_week_training_review=Your current training schedule shows a preference for workouts on Wednesdays and long runs on Saturdays, aligning well with your goals. You typically include rest or lighter running days on Sundays and incorporate moderate pacing throughout the week. To enhance marathon preparation, incorporating specific speed work on Wednesdays and ensuring a consistent long run on Saturdays will be beneficial., weekly_mileage_target=For Week 35, let's aim to slightly increase your mileage while still maintaining a conservative approach to ensure you're building safely. Target a total weekly mileage of around 46-47 miles, with your long run being approximately 16.5-17 miles. This progression aligns closely with your recent training pattern, allowing for a small but manageable increase that should keep you on t

In [8]:
mock_user = get_user(os.environ["JAMIES_ATHLETE_ID"])
response = run_mid_week_update_process_wrapped(mock_user)
print(response)

athlete_id=98390356 token still valid until 2024-08-28 20:18:36+00:00
athlete_id=98390356 token still valid until 2024-08-28 20:18:36+00:00
TrainingWeekWithPlanning(planning=The target for the week is to run a total of 46.0 miles. The client has already run 10.04 miles, leaving 35.96 miles to cover. The current plan adds up to 34.0 miles, which means we need to add 1.96 miles to the plan. There are five sessions left: a speed workout on Wednesday, a rest day on Thursday, an easy run on Friday, a long run on Saturday, and an easy run on Sunday. The rest day on Thursday does not contribute mileage. To distribute the extra mileage, we can slightly extend the length of the Friday easy run and the Sunday easy run, as the client may prefer not to add to the speed workout or long run. Currently, the Friday run is planned at 5.0 miles and the Sunday run at 4.0 miles. We will add 1.0 mile to the Friday run and 0.96 miles to the Sunday run. The adjusted plan is as follows: Wednesday speed workou

# Training week accuracy

In [6]:
path = 'tests/artifacts/new_training_week_errors'
files = os.listdir(path)

for file in sorted(files):
    print(f"Loading {file}")
    with open(os.path.join(path, file), "r") as f:
        results = json.load(f)
    print(sum(result['error'] for result in results) / len(results))

Loading 2024-08-28_16-47-43.json
1.97
Loading 2024-08-28_17-27-11.json
1.95
Loading 2024-08-28_17-41-17.json
1.95
Loading 2024-08-29_17-37-07.json
0.75
Loading 2024-08-29_17-45-21.json
0.95
Loading 2024-08-29_17-54-20.json
0.425
Loading 2024-08-29_18-09-32.json
0.475


In [15]:
for result in results:
    print(result['user']['email'])
    print("target:", result['recommended_weekly_mileage'])
    print("actual:", result['generated_weekly_mileage'])
    print("error:", result['error'])
    print(result['planning'])
    print("." * 100)

voynow99@gmail.com
target: 46-47
actual: 46.5
error: 0.0
Planning(weekly_mileage_target=46-47 miles, long_run_distance=17.0, remaining_weekly_mileage=46.5 (midpoint of 46-47 range) - 17 = 29.5, remaining_weekly_mileage_planning=Start by allocating mileage to the key days: Wednesday and Saturday. Since we already have 17 miles on Saturday, we have 29.5 miles left to distribute across the remaining days. Monday: 5 miles (easy pace), Tuesday: 6 miles (moderate pace), Wednesday: 7 miles (speed workout), Thursday: 5 miles (recovery run), Friday: rest day, Sunday: 6.5 miles (easy pace with strides). Double check: 5 + 6 + 7 + 5 + 6.5 = 29.5.)
....................................................................................................
jaredpalek8@gmail.com
target: 20 to 22 miles
actual: 23.0
error: 1.0
Planning(weekly_mileage_target=20 to 22 miles, long_run_distance=10.0, remaining_weekly_mileage=(20 - 10) to (22 - 10), remaining_weekly_mileage_planning=With a total weekly mileage targ